In [33]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

## Parametri

In [34]:
folder = os.path.join("..","processed_data")
savefolder = 'windowed_data'
tw = [2015,2022]
windowsize = 3

## Caricamento e utils

In [35]:
def temporalmerge(df1, temporaldf, columns):
    for prof in tqdm( df1.ID.unique() ):
        df2 = df1
        idx = df1.loc[df1.ID == prof].index
        temp = temporaldf.loc[temporaldf.ID == prof]

        for column in columns:
            columndict = {}

            for index,row in temp.iterrows():
                #each row is a year
                try:
                    columndict[row['anno']] = row[column]
                except:
                    columndict[row['year']] = row[column]

            for year in sorted( columndict.keys() ):
                df2.loc[idx, column +  "_" + str(year)] = columndict[year]

    df2.fillna(0, inplace=True)

    for col in df2.columns:
        try:
            if 'mean' not in col:
                df2[col] = df2[col].astype(int)
        except:
            continue

    return df2

In [36]:
settoriscientifici = pd.read_csv(os.path.join("..","data","settori_scientifici.csv"), index_col = 0).reset_index(drop=True)

In [37]:
docenti = pd.read_csv(os.path.join(folder,'docenti.csv'), index_col = 0)
afferisce = pd.read_csv(os.path.join(folder,'afferisce.csv'), index_col = 0)
ateneo = pd.read_csv(os.path.join(folder,'ateneo.csv'), index_col = 0)
fascia = pd.read_csv(os.path.join(folder,'fascia.csv'), index_col = 0)
lavora = pd.read_csv(os.path.join(folder,'lavora.csv'), index_col = 0)
ruolo = pd.read_csv(os.path.join(folder,'ruolo.csv'), index_col = 0)
scopus_static = pd.read_csv(os.path.join(folder,'scopus_static.csv'), index_col = 0)
scopus_yby = pd.read_csv(os.path.join(folder,'scopus_yby.csv'), index_col = 0)
settore = pd.read_csv(os.path.join(folder,'settore.csv'), index_col = 0)
settoreconcorsuale = pd.read_csv(os.path.join(folder,'settoreconcorsuale.csv'), index_col = 0)
strutturaafferenza = pd.read_csv(os.path.join(folder,'strutturaafferenza.csv'), index_col = 0)


## Costruzione DataFrame Italy

In [38]:
f = lambda x: str( x[1].split(' ')[0])
settoriscientifici['S.S.D.'] = settoriscientifici.apply(f, axis=1)
settoriscientifici = settoriscientifici[['Cod_AreaCUN','S.S.D.']]

In [39]:
docenti = docenti.merge(settoriscientifici)

In [40]:
lavora.head()

,ID,ateneo ID,anno
0,0,0,2019
1,0,0,2018
2,0,0,2022
3,0,0,2020
4,0,0,2021


In [41]:
lavora = lavora.drop(columns=['ateneo ID'])
lavmin = lavora.groupby(['ID'])['anno'].min().reset_index()
lavmin.rename(columns={'anno':'inizio'},inplace=True)
lavmax = lavora.groupby(['ID'])['anno'].max().reset_index()
lavmax.rename(columns={'anno':'fine'},inplace=True)
lavdur = lavora.groupby(['ID'])['anno'].count().reset_index()
lavdur.rename(columns={'anno':'durata'},inplace=True)

In [42]:
lavora = lavmin.merge(lavmax)
lavora = lavora.merge(lavdur)

In [43]:
lavora

,ID,inizio,fine,durata
0,0,2012,2022,11
1,1,2012,2022,11
2,2,2012,2022,11
3,3,2019,2022,4
4,4,2012,2022,11
...,...,...,...,...
84724,84724,2014,2016,3
84725,84725,2016,2017,2
84726,84726,2014,2015,2
84727,84727,2015,2015,1


In [44]:
lavora = lavora.loc[(lavora.inizio <= tw[0]) & (lavora.fine >= tw[1]) & (lavora.durata >= (tw[1]-tw[0]+1))]
lavora

,ID,inizio,fine,durata
0,0,2012,2022,11
1,1,2012,2022,11
2,2,2012,2022,11
4,4,2012,2022,11
5,5,2012,2022,11
...,...,...,...,...
84682,84682,2012,2022,10
84684,84684,2012,2022,11
84685,84685,2014,2022,9
84686,84686,2014,2022,9


In [45]:
docenti = docenti.merge(lavora)
#docenti

In [46]:
docenti.groupby('Cod_AreaCUN').count()

,Cognome e Nome,Genere,S.S.D.,Surname,First_Name,ID,inizio,fine,durata
Cod_AreaCUN,,,,,,,,,
1,2288,2288,2288,2288,2288,2288,2288,2288,2288
2,1488,1488,1488,1488,1488,1488,1488,1488,1488
3,2151,2151,2151,2151,2151,2151,2151,2151,2151
4,775,775,775,775,775,775,775,775,775
5,3358,3358,3358,3358,3358,3358,3358,3358,3358
6,5756,5756,5756,5756,5756,5756,5756,5756,5756
7,2372,2372,2372,2372,2372,2372,2372,2372,2372
8,2641,2641,2641,2641,2641,2641,2641,2641,2641
9,4299,4299,4299,4299,4299,4299,4299,4299,4299


Aggiungiamo i dati di SCOPUS

In [47]:
#docentiscopus = docenti.merge(scopus_static)

#total = docenti.merge(scopus_static)

total = docenti
total = temporalmerge(total,scopus_yby, ['papers', 'citations'])


  0%|          | 0/39950 [00:00<?, ?it/s]/var/folders/1f/dn5t35316d5b0gv0v6mcj6sm0000gn/T/ipykernel_3773/398045178.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df2.loc[idx, column +  "_" + str(year)] = columndict[year]
/var/folders/1f/dn5t35316d5b0gv0v6mcj6sm0000gn/T/ipykernel_3773/398045178.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df2.loc[idx, column +  "_" + str(year)] = columndict[year]
/var/folders/1f/dn5t35316d5b0gv0v6mcj6sm0000gn/T/ipykernel_3773/398045178.py:18: PerformanceWarning: DataFrame is highly 

In [48]:
r = ruolo.merge(fascia)
total = temporalmerge(total, r, ['fascia_id'])
gendict = {'F':0, 'M':1}
total['sex'] = total['Genere']
total.replace( {"sex":gendict}, inplace=True )

  0%|          | 0/39950 [00:00<?, ?it/s]

100%|██████████| 39950/39950 [01:26<00:00, 461.96it/s]
/var/folders/1f/dn5t35316d5b0gv0v6mcj6sm0000gn/T/ipykernel_3773/3813587996.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  total['sex'] = total['Genere']


## Anonimizzazione

In [49]:
total.drop(columns = ['Cognome e Nome', 'Genere', 'Surname', 'First_Name', 'ID', 'S.S.D.'], inplace=True)

In [50]:
total = total.loc[total.Cod_AreaCUN.isin([1,9])]

In [51]:
total.drop(columns=['Cod_AreaCUN'], inplace=True)

/var/folders/1f/dn5t35316d5b0gv0v6mcj6sm0000gn/T/ipykernel_3773/1059717045.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total.drop(columns=['Cod_AreaCUN'], inplace=True)


In [52]:
total.head()

,inizio,fine,durata,papers_1937,papers_1947,papers_1948,papers_1949,papers_1950,papers_1951,papers_1952,...,fascia_id_2014,fascia_id_2015,fascia_id_2016,fascia_id_2017,fascia_id_2018,fascia_id_2019,fascia_id_2020,fascia_id_2021,fascia_id_2022,sex
9553,2012,2022,11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
9554,2012,2022,11,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
9555,2012,2022,11,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,2,2,0
9556,2012,2022,11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9557,2012,2022,11,0,0,0,0,0,0,0,...,0,1,1,1,1,2,2,2,2,1


## Sliding Windows

In [53]:
total.to_csv("total_not_anon.csv")

In [54]:
total

,inizio,fine,durata,papers_1937,papers_1947,papers_1948,papers_1949,papers_1950,papers_1951,papers_1952,...,fascia_id_2014,fascia_id_2015,fascia_id_2016,fascia_id_2017,fascia_id_2018,fascia_id_2019,fascia_id_2020,fascia_id_2021,fascia_id_2022,sex
9553,2012,2022,11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
9554,2012,2022,11,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
9555,2012,2022,11,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,2,2,0
9556,2012,2022,11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9557,2012,2022,11,0,0,0,0,0,0,0,...,0,1,1,1,1,2,2,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39918,2012,2022,11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39919,2012,2022,11,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
39920,2012,2022,11,0,0,0,0,0,0,0,...,1,1,1,1,1,2,2,2,2,0
39921,2012,2022,11,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,1


In [55]:
total.rename(columns={'durata':'anni_servizio'}, inplace=True)
total.drop(columns={'inizio','fine'}, inplace=True)

/var/folders/1f/dn5t35316d5b0gv0v6mcj6sm0000gn/T/ipykernel_3773/3598284778.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total.rename(columns={'durata':'anni_servizio'}, inplace=True)
/var/folders/1f/dn5t35316d5b0gv0v6mcj6sm0000gn/T/ipykernel_3773/3598284778.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total.drop(columns={'inizio','fine'}, inplace=True)


In [56]:
total

,anni_servizio,papers_1937,papers_1947,papers_1948,papers_1949,papers_1950,papers_1951,papers_1952,papers_1953,papers_1954,...,fascia_id_2014,fascia_id_2015,fascia_id_2016,fascia_id_2017,fascia_id_2018,fascia_id_2019,fascia_id_2020,fascia_id_2021,fascia_id_2022,sex
9553,11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
9554,11,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
9555,11,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,2,2,0
9556,11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9557,11,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,2,2,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39918,11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39919,11,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
39920,11,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,2,2,2,2,0
39921,11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,1


In [57]:
df_yby = {}

In [58]:
for year in reversed(range( tw[0]+windowsize, tw[1]+1) ):
    yearstoconsider = list(range(year-windowsize,year+1))
    startingyear = 1937

    todrop = []
    ## Remove citations and publications
    for year2 in range(startingyear,2023):
        if year2 not in yearstoconsider:
            if 'papers_' + str(year2) in total.columns:
                todrop.append("papers_" + str(year2))
                todrop.append("citations_" + str(year2))

    #Remove fascia_id except current year
    for year2 in range(startingyear,2023):
        if year2 != year:
            if 'fascia_id_' + str(year2) in total.columns:
                todrop.append("fascia_id_" + str(year2))

    # Sum papers and citations up until year
    startingyear = 1937
    total_papers = total['papers_' + str(startingyear)]
    total_citations = total['citations_' + str(startingyear)]
    for yyyy in range(startingyear+1,year+1):
        if 'papers_' + str(yyyy) in total.columns:
            total_papers = total_papers + total['papers_' + str(yyyy)]
            total_citations = total_citations + total['citations_' + str(yyyy)]
    

    yb = total.drop(columns=todrop)
    yb.rename(columns={'fascia_id_' + str(year) : 'fascia_id'}, inplace=True)
    yb['Total_papers'] = total_papers
    yb['Total_citations'] = total_citations

    df_yby[year] = yb

In [59]:
df_yby[2022].columns

Index(['anni_servizio', 'papers_2019', 'papers_2020', 'papers_2021',
       'papers_2022', 'citations_2019', 'citations_2020', 'citations_2021',
       'citations_2022', 'fascia_id', 'sex', 'Total_papers',
       'Total_citations'],
      dtype='object')

In [60]:

for y in df_yby.keys():
    df = df_yby[y]
    df.rename(columns={'papers_' + str(y-3): 'papers_-3', 'papers_' + str(y-2): 'papers_-2', 'papers_' + str(y-1): 'papers_-1', 'papers_' + str(y): 'papers_0',
    'citations_' + str(y-3): 'citations_-3', 'citations_' + str(y-2): 'citations_-2', 'citations_' + str(y-1): 'citations_-1', 'citations_' + str(y): 'citations_0'},inplace=True)


In [61]:
df_yby[2022]

,anni_servizio,papers_-3,papers_-2,papers_-1,papers_0,citations_-3,citations_-2,citations_-1,citations_0,fascia_id,sex,Total_papers,Total_citations
9553,11,2,1,1,1,19,3,4,0,1,1,39,660
9554,11,3,2,1,1,24,20,3,0,1,1,48,2144
9555,11,6,4,4,5,156,21,35,1,2,0,109,2626
9556,11,7,1,4,1,32,15,13,5,0,1,41,450
9557,11,24,4,12,9,329,94,182,3,2,1,193,8575
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39918,11,3,3,4,2,12,24,8,0,0,0,33,330
39919,11,1,1,0,0,5,1,0,0,1,1,28,96
39920,11,4,2,3,2,17,2,3,1,2,0,88,963
39921,11,5,2,3,2,29,7,3,3,1,1,49,372


In [62]:
for year in df_yby.keys():
    df_yby[year].to_csv(os.path.join(savefolder,str(year) + ".csv"))